In [201]:
import sagemaker
import boto3
from sagemaker import get_execution_role

region = boto3.Session().region_name

session = sagemaker.Session()

# You can modify the following to use a bucket of your choosing
bucket = "sagemaker-eu-west-3-177285995440"
prefix = "V7"

role = get_execution_role()

# This is the client we will use to interact with SageMaker AutoPilot
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [202]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

In [203]:
!aws s3 cp s3://bing-training-api-easy-prospects/V1/dataset.csv ./

download: s3://bing-training-api-easy-prospects/V1/dataset.csv to ./dataset.csv


In [204]:
dataset = pd.read_csv("./dataset.csv")
pd.set_option("display.max_columns", 500)
dataset = dataset.drop(columns=["target_dom"])
dataset


,url_rank,url,full_url,snippet,language,domain,name,isfamilyfriendly,isnavigational,siren,siret,nom_raison_sociale,numero_voie,type_voie,libelle_voie,code_postal,libelle_commune,departement,geo_adresse,activite_principale,libelle_activite_principale_entreprise,tranche_effectif_salarie,categorie_entreprise,latitude,longitude,dom_lev_score,max_rs_n_gram_matching_dom,max_len_string_match,dom_ville_score,dom_dep_score,snippet_score,lang,adresse,cat
0,11,https://www.hotel-seminaire.com/,http://www.hotel-seminaire.com/hotels-seminair...,Découvrez Garden Beach Hôtel situé à Antibes e...,fr,hotel-seminaire.com,Garden Beach Hôtel Location Salle de Séminaire...,True,False,351863089,35186308900022,GARDEN BEACH HOTEL,13.0,BD,EDOUARD BAUDOIN,6160.0,ANTIBES,06,13 Boulevard Edouard Baudoin 06160 Antibes,5510Z,Hôtels et hébergement similaire,20 à 49 salariés,PME,43.567434,7.113465,0.071429,1.632734,0.055556,0,0,0.241003,1,13 BD EDOUARD BAUDOIN,0
1,14,https://www.french-corporate.com/,https://french-corporate.com/french-corporate....,La société COBALP est situé au 2 RUE DU LEVRAY...,fr,french-corporate.com,French Corporate - COBALP,True,False,351054580,35105458000037,COBALP,2.0,RUE,DU LEVRAY,74960.0,ANNECY,74,2 R du Levray 74600 Annecy,7490A,Activité des économistes de la construction,20 à 49 salariés,PME,45.890421,6.083083,-1.000000,0.000000,0.333333,0,0,0.147714,1,2 RUE DU LEVRAY,0
2,13,https://www.ellisphere.fr/,https://www.ellisphere.fr/dirigeant/PAVELIC-VI...,Dirigeant Vincent PAVELIC - Gérant de l&#39;&#...,fr,ellisphere.fr,Vincent PAVELIC - Gérant de l&#39;&#39;entrepr...,True,False,323005090,32300509000036,PAVELIC IMMOBILIER,2.0,PL,ROYALE,64000.0,PAU,64,2 Place Royale 64000 Pau,6810Z,Activités des marchands de biens immobiliers,effectif non connu,PME,43.293776,-0.370589,-0.300000,0.000000,0.076923,0,0,0.261457,1,2 PL ROYALE,0
3,3,https://www.lefigaro.fr/,https://entreprises.lefigaro.fr/deprez-guignot...,"deprez, guignot &amp; associes sociéte civile ...",fr,lefigaro.fr,"Deprez Guignot &amp; Associes DDG (Paris, 7500...",True,False,342119047,34211904700021,DEPREZ GUIGNOT & ASSOCIES,21.0,RUE,CLEMENT MAROT,75008.0,PARIS,75,21 Rue Clément Marot 75008 Paris,6910Z,Activités juridiques,20 à 49 salariés,PME,48.868263,2.302238,-1.000000,0.000000,0.090909,0,0,0.264184,1,21 RUE CLEMENT MAROT,0
4,6,https://www.kompass.com/,https://fr.kompass.com/c/mtb/fr8382016/,FR8382016. Présentation - MTB. Société spécial...,fr,kompass.com,"Mtb - Clichy 92110 (Hauts-de-seine), 7 Pl Mart...",True,False,333883221,33388322100058,MTB,7.0,PL,MARTYRS DE L'OCCUPATION,92110.0,CLICHY,92,7 Place Martyrs de l’Occupation 92110 Clichy,8299Z,Autres activités de soutien aux entreprises n....,6 à 9 salariés,PME,48.901438,2.306286,-1.000000,0.000000,0.333333,0,0,0.317859,1,7 PL MARTYRS DE L'OCCUPATION,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11900,6,https://www.trackmotor.fr/,https://trackmotor.fr/,Concessionnaire moto multi marqueset vente de ...,fr,trackmotor.fr,Concessionnaire moto multi marques ... - Track...,True,False,881860795,88186079500015,PRINTOO.FR,3.0,RUE,THALES,17440.0,AYTRE,17,3bis Rue Thales 17440 Aytré,7311Z,Activités des agences de publicité,effectif non connu,NaN,46.127033,-1.092468,0.222222,0.000000,0.200000,0,0,0.121270,1,3 RUE THALES,0
11901,2,https://www.pagesjaunes.fr/,https://www.pagesjaunes.fr/siret/88193766800013,Conex Sante Occimed (SIRET 88193766800013) à V...,fr,pagesjaunes.fr,Conex Sante Occimed à Villeneuve du Latou : SI...,True,False,881937668,88193766800013,CONEX SANTE,NaN,LD,TATONNE,9130.0,VILLENEUVE DU LATOU,09,Tatonne 09130 Villeneuve-du-Latou,6201Z,Programmation informatique,effectif non connu,NaN,43.189172,1.437269,0.200000,0.000000,0.181818,0,0,0.130429,1,LD TATONNE,0
11902,10,https://www.plages.tv/,https://www.plages.tv/detail/plage-du-ruhl-nic...,Notre avis sur la Plage du Ruhl. La plage Ruhl...,fr,plages.tv,Plage du Ruhl Nice (06) Alpes-

In [205]:
train_data = dataset.sample(frac=0.8, random_state=200)

test_data = dataset.drop(train_data.index)

test_data_no_target = test_data.drop(columns=["cat"])

In [206]:
train_file = "train_data.csv"
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print("Train data uploaded to: " + train_data_s3_path)

test_file = "test_data.csv"
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print("Test data uploaded to: " + test_data_s3_path)

Train data uploaded to: s3://sagemaker-eu-west-3-177285995440/V7/train/train_data.csv
Test data uploaded to: s3://sagemaker-eu-west-3-177285995440/V7/test/test_data.csv


In [208]:
bucket = "sagemaker-eu-west-3-177285995440"
input_data_config = [
    {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
            }
        },
        "TargetAttributeName": "cat",
    }
]

output_data_config = {"S3OutputPath": "s3://{}/{}/output".format(bucket, prefix)}


problem_type = 'BinaryClassification'

job_objective = { 'MetricName': 'F1' }

In [209]:
from time import gmtime, strftime, sleep

timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())

auto_ml_job_name = "bing-automl" + timestamp_suffix
print("AutoMLJobName: " + auto_ml_job_name)

sm.create_auto_ml_job(
    AutoMLJobName=auto_ml_job_name,
    InputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLJobConfig={"CompletionCriteria": {"MaxCandidates": 15}},
    RoleArn=role,
    AutoMLJobObjective=job_objective,
    ProblemType=problem_type,
)

AutoMLJobName: bing-automl24-16-14-47


{'AutoMLJobArn': 'arn:aws:sagemaker:eu-west-3:177285995440:automl-job/bing-automl24-16-14-47',
 'ResponseMetadata': {'RequestId': '87f23f7c-9aa9-4e67-a9d1-61f0cbafbc1d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '87f23f7c-9aa9-4e67-a9d1-61f0cbafbc1d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '93',
   'date': 'Mon, 24 Jan 2022 16:14:47 GMT'},
  'RetryAttempts': 0}}

In [210]:
%store auto_ml_job_name

Stored 'auto_ml_job_name' (str)


In [212]:
print("JobStatus - Secondary Status")
print("------------------------------")


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"])
job_run_status = describe_response["AutoMLJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response["AutoMLJobStatus"]

    print(
        describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"]
    )
    sleep(30)

JobStatus - Secondary Status
------------------------------
Completed - Completed


In [213]:
# print(describe_response)
print(describe_response["AutoMLJobArtifacts"]["CandidateDefinitionNotebookLocation"])
print(describe_response["AutoMLJobArtifacts"]["DataExplorationNotebookLocation"])

candidate_nbk = describe_response["AutoMLJobArtifacts"]["CandidateDefinitionNotebookLocation"]
data_explore_nbk = describe_response["AutoMLJobArtifacts"]["DataExplorationNotebookLocation"]

s3://sagemaker-eu-west-3-177285995440/V7/output/bing-automl24-16-14-47/sagemaker-automl-candidates/bing-automl24-16-14-47-pr-1-0abc8fb308764c12a741167fdc76bf8357e/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb
s3://sagemaker-eu-west-3-177285995440/V7/output/bing-automl24-16-14-47/sagemaker-automl-candidates/bing-automl24-16-14-47-pr-1-0abc8fb308764c12a741167fdc76bf8357e/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb


In [214]:
def split_s3_path(s3_path):
    path_parts = s3_path.replace("s3://", "").split("/")
    bucket = path_parts.pop(0)
    key = "/".join(path_parts)
    return bucket, key


s3_bucket, candidate_nbk_key = split_s3_path(candidate_nbk)
_, data_explore_nbk_key = split_s3_path(data_explore_nbk)

print(s3_bucket, candidate_nbk_key, data_explore_nbk_key)

session.download_data(path="./", bucket=s3_bucket, key_prefix=candidate_nbk_key)

session.download_data(path="./", bucket=s3_bucket, key_prefix=data_explore_nbk_key)

sagemaker-eu-west-3-177285995440 V7/output/bing-automl24-16-14-47/sagemaker-automl-candidates/bing-automl24-16-14-47-pr-1-0abc8fb308764c12a741167fdc76bf8357e/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb V7/output/bing-automl24-16-14-47/sagemaker-automl-candidates/bing-automl24-16-14-47-pr-1-0abc8fb308764c12a741167fdc76bf8357e/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb


In [215]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)["BestCandidate"]
best_candidate_name = best_candidate["CandidateName"]
print(best_candidate)
print("\n")
print("CandidateName: " + best_candidate_name)
print(
    "FinalAutoMLJobObjectiveMetricName: "
    + best_candidate["FinalAutoMLJobObjectiveMetric"]["MetricName"]
)
print(
    "FinalAutoMLJobObjectiveMetricValue: "
    + str(best_candidate["FinalAutoMLJobObjectiveMetric"]["Value"])
)

{'CandidateName': 'bing-automl24-16-14-47mGiu2zG1Ie-012-fc09288f', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:binary_f_beta', 'Value': 0.8678621649742126}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:eu-west-3:177285995440:processing-job/bing-automl24-16-14-47-db-1-75f97ec5100445e2b166485cf13c70cedc4', 'CandidateStepName': 'bing-automl24-16-14-47-db-1-75f97ec5100445e2b166485cf13c70cedc4'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:eu-west-3:177285995440:training-job/bing-automl24-16-14-47-dpp9-1-bbea5a41136e4efcbc7eb9b6d487c2b23', 'CandidateStepName': 'bing-automl24-16-14-47-dpp9-1-bbea5a41136e4efcbc7eb9b6d487c2b23'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:eu-west-3:177285995440:transform-job/bing-automl24-16-14-47-dpp9-csv-1-9ba0e05ddad043b7acc07961aa628', 'Candidate

In [216]:
# sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
# sm.list_auto_ml_jobs()
sm_dict = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name)

In [217]:
for item in sm_dict["Candidates"]:
    print(item["CandidateName"], item["FinalAutoMLJobObjectiveMetric"])
    print(item["InferenceContainers"][1]["Image"], "\n")

bing-automl24-16-14-47mGiu2zG1Ie-014-adb23de8 {'MetricName': 'validation:f1_binary', 'Value': 0.8663600087165833}
659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-xgboost:1.2-2-cpu-py3 

bing-automl24-16-14-47mGiu2zG1Ie-015-53c603a6 {'MetricName': 'validation:f1_binary', 'Value': 0.7919800281524658}
659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-xgboost:1.2-2-cpu-py3 

bing-automl24-16-14-47mGiu2zG1Ie-013-1970c716 {'MetricName': 'validation:binary_f_beta', 'Value': 0.7991360425949097}
749696950732.dkr.ecr.eu-west-3.amazonaws.com/linear-learner:latest 

bing-automl24-16-14-47mGiu2zG1Ie-012-fc09288f {'MetricName': 'validation:binary_f_beta', 'Value': 0.8678621649742126}
749696950732.dkr.ecr.eu-west-3.amazonaws.com/mxnet-algorithms:inference-cpu 

bing-automl24-16-14-47mGiu2zG1Ie-011-4a5d7fa8 {'MetricName': 'validation:f1_binary', 'Value': 0.7711499929428101}
659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-xgboost:1.2-2-cpu-py3 

bing-automl24-16-14-47mGiu2zG1Ie-006-b

In [218]:
timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
model_name = best_candidate_name + timestamp_suffix + "-model"
Containers=best_candidate["InferenceContainers"]
Containers[1]['Environment'].update({'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability'})
Containers[2]['Environment'].update({'SAGEMAKER_INFERENCE_INPUT': 'predicted_label, probability'})
Containers[2]['Environment'].update({'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability'})

model_arn = sm.create_model(
    Containers=Containers,
    ModelName=model_name,
    ExecutionRoleArn=role
)

epc_name = best_candidate_name + timestamp_suffix + "-epc"
ep_config = sm.create_endpoint_config(
    EndpointConfigName=epc_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.t2.medium",
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "VariantName": "main",
        }
    ],
)

ep_name = best_candidate_name + timestamp_suffix + "-ep"
create_endpoint_response = sm.create_endpoint(EndpointName=ep_name, EndpointConfigName=epc_name)

In [219]:
sm.get_waiter("endpoint_in_service").wait(EndpointName=ep_name)

In [226]:
from io import StringIO
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
import time

print("ep_name : ", ep_name)

def predict_website(df, seuil):
    
    predictor = Predictor(
                        endpoint_name=ep_name,
                        sagemaker_session=session,
                        serializer=CSVSerializer(),
                        deserializer=CSVDeserializer(),
                        )
    test_data_inference = df
    print(test_data_inference.columns)
    prediction = predictor.predict(test_data_inference.to_csv(sep=",", header=False, index=False))
    prediction_df = pd.DataFrame(prediction)
    prediction_df_labels = prediction_df[0].astype(int)
    prediction_df_probas = prediction_df[1].astype(float)
    idmax = prediction_df_probas.idxmax()
    if prediction_df_probas[idmax] > seuil:
        return df.iloc[idmax]

ep_name :  bing-automl24-16-14-47mGiu2zG1Ie-012-fc09288f24-18-17-32-ep


In [227]:
# Remove the target column from the test data
test_data_inference = test_data.drop("cat", axis=1)

print(predict_website(test_data_inference, 0.5))

Index(['url_rank', 'url', 'full_url', 'snippet', 'language', 'domain', 'name',
       'isfamilyfriendly', 'isnavigational', 'siren', 'siret',
       'nom_raison_sociale', 'numero_voie', 'type_voie', 'libelle_voie',
       'code_postal', 'libelle_commune', 'departement', 'geo_adresse',
       'activite_principale', 'libelle_activite_principale_entreprise',
       'tranche_effectif_salarie', 'categorie_entreprise', 'latitude',
       'longitude', 'dom_lev_score', 'max_rs_n_gram_matching_dom',
       'max_len_string_match', 'dom_ville_score', 'dom_dep_score',
       'snippet_score', 'lang', 'adresse'],
      dtype='object')
url_rank                                                                                  2
url                                                         https://www.maisonpassive71.fr/
full_url                                            https://www.maisonpassive71.fr/contact/
snippet                                   Maison Passive 71. 2393 rue Varmancourt. 71450...
lan

In [ ]:
def predict_website(df, seuil):
    predictor = Predictor(
                        endpoint_name=ep_name,
                        sagemaker_session=session,
                        serializer=CSVSerializer(),
                        deserializer=CSVDeserializer(),
                        )
    test_data_inference = df
    prediction = predictor.predict(test_data_inference.to_csv(sep=",", header=False, index=False))
    prediction_df = pd.DataFrame(prediction)
    prediction_df_labels = prediction_df[0].astype(int)
    prediction_df_probas = prediction_df[1].astype(float)
    idmax = prediction_df_probas.idxmax()
    if prediction_df_probas[idmax] > seuil:
        return df.iloc[idmax]


In [163]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(test_data.reset_index()["cat"], prediction_df_labels)
precision = precision_score(test_data.reset_index()["cat"], prediction_df_labels, pos_label=1)

recall = recall_score(
    test_data.reset_index()["cat"], prediction_df_labels, pos_label=1, average="binary"
)
f1 = f1_score(test_data.reset_index()["cat"], prediction_df_labels, pos_label=1)
confusion = confusion_matrix(test_data.reset_index()["cat"], prediction_df_labels)

print("Accuracy: {}".format(accuracy))
print("Precision: {}".format(precision))
print("Recall: {}".format(recall))
print("F1: {}".format(f1))
print("Confusion Matrix: {}".format(confusion))

Accuracy: 0.9802603947921041
Precision: 0.9024390243902439
Recall: 0.9061224489795918
F1: 0.9042769857433809
Confusion Matrix: [[2112   24]
 [  23  222]]
